<a href="https://colab.research.google.com/github/hay-k/CohortProject_2020_w4g4/blob/master/Project_4_Ising_Annealer/Car_Rental_Schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Note you need to do the pip install and git clone only once!
#That is, at the begining of your runtime on the Google Colab.

#Car Rental Schedule
based on the annealing for the Travelling Salesman Problem
https://github.com/perrygeo/simanneal

In [ ]:
 #!git clone https://github.com/perrygeo/simanneal.git

Cloning into 'simanneal'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 312 (delta 15), reused 23 (delta 8), pack-reused 275
Receiving objects: 100% (312/312), 819.28 KiB | 7.25 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [ ]:
#!cd ./simanneal/ && pip install -e .

Obtaining file:///content/simanneal
  Found existing installation: simanneal 0.5.0
    Uninstalling simanneal-0.5.0:
      Successfully uninstalled simanneal-0.5.0
  Running setup.py develop for simanneal


In [ ]:
#pip install -e git+https://github.com/perrygeo/simanneal.git  # latest from github
#!pip install simanneal  # from pypi

!pip show simanneal  # from pypi

Name: simanneal
Version: 0.5.0
Summary: Simulated Annealing in Python
Home-page: https://github.com/perrygeo/simanneal
Author: Matthew Perry
Author-email: perrygeo@gmail.com
License: BSD
Location: /content/simanneal
Requires: 
Required-by: 


In [ ]:
import sys
f_loc="./simanneal/tests/"
if f_loc not in sys.path: sys.path.append(f_loc)

In [ ]:
import random
import sys
import time

from helper import distance, cities, distance_matrix
from simanneal import Annealer

if sys.version_info.major >= 3:  # pragma: no cover
    from io import StringIO
else:
    from StringIO import StringIO

In [ ]:
%%time
#see ./simanneal/examples/salesman.py
# -*- coding: utf-8 -*-
from __future__ import print_function
import math
import random
from simanneal import Annealer


def distance(a, b):
    """Calculates distance between two latitude-longitude coordinates."""
    R = 3963  # radius of Earth (miles)
    lat1, lon1 = math.radians(a[0]), math.radians(a[1])
    lat2, lon2 = math.radians(b[0]), math.radians(b[1])
    return math.acos(math.sin(lat1) * math.sin(lat2) +
                     math.cos(lat1) * math.cos(lat2) * math.cos(lon1 - lon2)) * R


class TravellingSalesmanProblem(Annealer):

    """Test annealer with a travelling salesman problem.
    """

    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, distance_matrix):
        self.distance_matrix = distance_matrix
        super(TravellingSalesmanProblem, self).__init__(state)  # important!

    def move(self):
        """Swaps two cities in the route."""
        # no efficiency gain, just proof of concept
        # demonstrates returning the delta energy (optional)
        initial_energy = self.energy()

        a = random.randint(0, len(self.state) - 1)
        b = random.randint(0, len(self.state) - 1)
        self.state[a], self.state[b] = self.state[b], self.state[a]

        return self.energy() - initial_energy

    def energy(self):
        """Calculates the length of the route."""
        e = 0
        for i in range(len(self.state)):
            e += self.distance_matrix[self.state[i-1]][self.state[i]]
        return e


if __name__ == '__main__':

    # latitude and longitude for the twenty largest U.S. cities
    cities = {
        'New York City': (40.72, 74.00),
        'Los Angeles': (34.05, 118.25),
        'Chicago': (41.88, 87.63),
        'Houston': (29.77, 95.38),
        'Phoenix': (33.45, 112.07),
        'Philadelphia': (39.95, 75.17),
        'San Antonio': (29.53, 98.47),
        'Dallas': (32.78, 96.80),
        'San Diego': (32.78, 117.15),
        'San Jose': (37.30, 121.87),
        'Detroit': (42.33, 83.05),
        'San Francisco': (37.78, 122.42),
        'Jacksonville': (30.32, 81.70),
        'Indianapolis': (39.78, 86.15),
        'Austin': (30.27, 97.77),
        'Columbus': (39.98, 82.98),
        'Fort Worth': (32.75, 97.33),
        'Charlotte': (35.23, 80.85),
        'Memphis': (35.12, 89.97),
        'Baltimore': (39.28, 76.62)
    }

    # initial state, a randomly-ordered itinerary
    init_state = list(cities.keys())
    random.shuffle(init_state)

    # create a distance matrix
    distance_matrix = {}
    for ka, va in cities.items():
        distance_matrix[ka] = {}
        for kb, vb in cities.items():
            if kb == ka:
                distance_matrix[ka][kb] = 0.0
            else:
                distance_matrix[ka][kb] = distance(va, vb)

    tsp = TravellingSalesmanProblem(init_state, distance_matrix)
    tsp.set_schedule(tsp.auto(minutes=0.2))
    # since our state is just a list, slice is the fastest way to copy
    tsp.copy_strategy = "slice"
    state, e = tsp.anneal()

    while state[0] != 'New York City':
        state = state[1:] + state[:1]  # rotate NYC to start

    print()
    print("%i mile route:" % e)
    print(" ➞  ".join(state))



 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.80000       6887.06     5.80%     0.00%     0:00:02    -1:59:59
 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.80000       6801.84     5.24%     0.00%     0:00:06     0:00:00


6801 mile route:
New York City ➞  Detroit ➞  Columbus ➞  Indianapolis ➞  Chicago ➞  San Francisco ➞  San Jose ➞  Los Angeles ➞  San Diego ➞  Phoenix ➞  San Antonio ➞  Austin ➞  Houston ➞  Fort Worth ➞  Dallas ➞  Memphis ➞  Jacksonville ➞  Charlotte ➞  Baltimore ➞  Philadelphia
CPU times: user 8.2 s, sys: 104 ms, total: 8.3 s
Wall time: 8.31 s


In [ ]:
cities

{'Austin': (30.27, 97.77),
 'Baltimore': (39.28, 76.62),
 'Charlotte': (35.23, 80.85),
 'Chicago': (41.88, 87.63),
 'Columbus': (39.98, 82.98),
 'Dallas': (32.78, 96.8),
 'Detroit': (42.33, 83.05),
 'Fort Worth': (32.75, 97.33),
 'Houston': (29.77, 95.38),
 'Indianapolis': (39.78, 86.15),
 'Jacksonville': (30.32, 81.7),
 'Los Angeles': (34.05, 118.25),
 'Memphis': (35.12, 89.97),
 'New York City': (40.72, 74.0),
 'Philadelphia': (39.95, 75.17),
 'Phoenix': (33.45, 112.07),
 'San Antonio': (29.53, 98.47),
 'San Diego': (32.78, 117.15),
 'San Francisco': (37.78, 122.42),
 'San Jose': (37.3, 121.87)}

In [ ]:
def make_itinerary(current_location,initial_state):

  if current_location not in initial_state:
    print("Add you current location to the initial_state!")
    return

  # initial state, a randomly-ordered itinerary
  init_state = initial_state
  random.shuffle(init_state)

  # populate your cities list with coordinate information
  my_cities={}
  for city in initial_state:
    if city in cities.keys():
      my_cities[city]=cities[city]
    else:
      print("No location coordinates for ",city)
      if city==current_location:
        print("Add your location coordinates!")
        return

  # create a distance matrix
  distance_matrix = {}
  for ka, va in my_cities.items():
      distance_matrix[ka] = {}
      for kb, vb in my_cities.items():
          if kb == ka:
              distance_matrix[ka][kb] = 0.0
          else:
              distance_matrix[ka][kb] = distance(va, vb)

  tsp = TravellingSalesmanProblem(init_state, distance_matrix)
  '''
  tsp.set_schedule(tsp.auto(minutes=0.2))
  # since our state is just a list, slice is the fastest way to copy
  tsp.copy_strategy = "slice"
  '''
  itinerary, miles = tsp.anneal()

  while itinerary[0] != current_location:
      itinerary = itinerary[1:] + itinerary[:1]  # rotate to start correctly

  print()
  print("%i mile route:" % miles)
  print(" ➞  ".join(itinerary))


In [ ]:
%%time
initial_state = ['New York City', 'Los Angeles', 'Philadelphia', 'Houston']
initial_state = list(cities.keys())
current_location= 'Los Angeles'

make_itinerary(current_location,initial_state)


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000       6845.62     5.00%     0.00%     0:00:02     0:00:00


6845 mile route:
Los Angeles ➞  San Jose ➞  San Francisco ➞  Fort Worth ➞  Dallas ➞  Memphis ➞  Indianapolis ➞  Chicago ➞  Detroit ➞  Columbus ➞  New York City ➞  Philadelphia ➞  Baltimore ➞  Charlotte ➞  Jacksonville ➞  Houston ➞  Austin ➞  San Antonio ➞  Phoenix ➞  San Diego
CPU times: user 1.56 s, sys: 34.3 ms, total: 1.59 s
Wall time: 1.62 s
